In [234]:
import pandas as pd
import requests
import random
import time
from time import sleep
from bs4 import BeautifulSoup
from random import randint

In [235]:
def Check_number_of_pages(link,n):
    # Parse HTML text
    for x in range(1,n):
        r = requests.get(f"{link}&page={x}")
        soup = BeautifulSoup(r.text, 'html.parser')
        products_check = soup.find_all('div',{'class':'product-item'})
        items = len(products_check)
        y = x
        if items == 0:
            pages = y-1
            print(f'{pages} is the final pages of this product')
            return pages 
        else:
            print(f'{y-1} is not the final pages of this product')
    return pages

In [241]:
import pandas as pd
import requests
import random
import time
from time import sleep
from bs4 import BeautifulSoup
from random import randint   
    
def crawling_product(url1,number_of_pages,pdn):
    """Scrape the page of product you want to
      Input: url to the webpage. Default: https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner
      Output: A list containing scraped data of all product & price
    """
    # Get parsed HTML
    dataall = []   
    x = 1
    while x <= number_of_pages:
        url=f"{url1}&page={x}"
        r = requests.get(url)
        # Parse HTML text
        soup = BeautifulSoup(r.text, 'html.parser')  
        # Find all products in the page
        products = soup.find_all('div',{'class':'product-item'}) 
        x += 1
        data = []
        # List containing data of all products 
        for product in products: 
            d = {'Product-id':'','Seller-id':'','Product-SKU':'','Product-Title':'','Product-Brand':'',
                 'URL':'','img_URL':'','Regular-Price':'','Final-Price':'','TIKI-NOW':'','Installment':'',
                 'Page':'','Location':''}
            #other optional : 'Rating':'',
            try:
                d['Product-id'] = product.a['data-id']
                d['Product-SKU'] = product['product-sku']
                d['Seller-id'] = product['data-seller-product-id']
                d['Product-Title'] = product.a['title']
                d['Product-Brand'] = product['data-brand']
                d['URL'] = (f"https://tiki.vn/{product.a['href']}")
                d['img_URL'] = product.a.div.span.img['src']
                Finalprice = product.find('span',{'class':'final-price'})
                d['Final-Price'] = Finalprice.text.strip(" \n, ...")
                #Rating = product.find('span',{'class':'rating-content'})
                #d['Rating'] = Rating.style.strip("width:")
                d['Location'] = products.index(product)
                d['Page'] = x-1
                #Let's Check for somethings deeper
                try: 
                    #Some products doesn't have regular price
                    Regularprice = product.find('span',{'class':'price-regular'})
                    d['Regular-Price'] = Regularprice.text
                    #Or somethings like Installment
                    Installment = product.find('span',{'class':'installment-price-v2'})
                    d['Installment'] = Installment.text
                    #TIKI-NOW - aka Unique Value of TIKI
                    tikinow = product.find('div',{'class':'badge-service'})
                    badge_check = tikinow.div.img['src']
                    if str(badge_check) == "https://salt.tikicdn.com/ts/upload/9f/32/dd/8a8d39d4453399569dfb3e80fe01de75.png":
                        d['TIKI-NOW'] = "YES"
                except:
                    d['Regular-Price'] = d['Final-Price']
                    d['Installment'] = None
                    d['TIKI-NOW'] = "NO"
                #Then we appending all the Data from this page:       
                data.append(d)   
            #If missing some info let's the syntax warning back.
            except:
                print("We got one product missing somes info!")
        #Then let's bring all data of products from one page to extend one data name "dataall"   
        dataall.extend(data)
        #Use pandas to DF the dataall, with colums keys is"dataall[0]""     
        full_products = pd.DataFrame(data = dataall, columns = dataall[0].keys())
        #Then convert it to CSV for saving & visualization and name it by "pdn" = product-name
        full_products.to_csv(f'./Full_Products_{pdn}.csv', index=False)
#     sleeptime = random.randint(2,3)
#     sleep(sleeptime)  
           
    return full_products

In [242]:
link ='https://tiki.vn/linh-kien-may-tinh-phu-kien-may-tinh/c8129/asus?src=c.8095.hamburger_menu_fly_out_bannerr'
crawling_product(link, Check_number_of_pages(link,60), 'lkmt')

0 is not the final pages of this product
1 is not the final pages of this product
2 is not the final pages of this product
3 is not the final pages of this product
4 is the final pages of this product


,Product-id,Seller-id,Product-SKU,Product-Title,Product-Brand,URL,img_URL,Regular-Price,Final-Price,TIKI-NOW,Installment,Page,Location
0,30667553,30667554,6182084225433,Bo Mạch Chủ Mainboard Asus EX-B365M-V5 - Hàng ...,Asus,https://tiki.vn//bo-mach-chu-mainboard-asus-ex...,https://salt.tikicdn.com/cache/280x280/ts/prod...,1.490.000đ ...,1.490.000đ ...,NO,None,1,0
1,55708956,55708986,6462317245408,Card màn hình VGA ASUS ROG-STRIX-RX5600XT-T6G-...,Asus,https://tiki.vn//crad-man-hinh-vga-asus-rog-st...,https://salt.tikicdn.com/cache/280x280/ts/prod...,10.990.000đ,8.548.000đ ...,YES,Trả góp 0%,1,1
2,54636907,54636908,6000770438373,Bo Mạch Chủ Mainboard ASUS ROG STRIX B460-I GA...,Asus,https://tiki.vn//bo-mach-chu-mainboard-asus-ro...,https://salt.tikicdn.com/cache/280x280/ts/prod...,3.648.000đ ...,3.648.000đ ...,NO,None,1,2
3,1911005,35240333,5136405096326,Card Màn Hình VGA ASUS PH-GTX1050TI 4GB 128bit...,Asus,https://tiki.vn//card-man-hinh-vga-asus-ph-gtx...,https://salt.tikicdn.com/cache/280x280/ts/prod...,5.390.000đ,3.050.000đ ...,YES,Trả góp 0%,1,3
4,23197333,23197334,6313909213506,Card Màn Hình VGA ASUS ROG STRIX GTX1650 O4G G...,Asus,https://tiki.vn//card-man-hinh-vga-asus-rog-st...,https://salt.tikicdn.com/cache/280x280/ts/prod...,5.990.000đ,4.378.000đ ...,YES,Trả góp 0%,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,54640648,54640649,1484402636507,Bo Mạch Chủ Mainboard ASUS TUF Z390-PLUS GAMIN...,Asus,https://tiki.vn//bo-mach-chu-mainboard-asus-tu...,https://salt.tikicdn.com/cache/280x280/ts/prod...,7.990.000đ,7.990.000đ,NO,None,4,17
162,35355486,36160929,6487915584933,Bo mạch chủ ASUS MAXIMUS VIII RANGER ATX Z170...,Asus,https://tiki.vn//bo-mach-chu-asus-maximus-viii...,https://salt.tikicdn.com/cache/280x280/ts/prod...,5.490.000đ,5.490.000đ,NO,None,4,18
163,28055369,28055370,3197624577135,Card Màn Hình VGA ASUS DUAL-GTX1660TI-6G GDDR6...,Asus,https://tiki.vn//card-man-hinh-vga-asus-dual-g...,https://salt.tikicdn.com/cache/280x280/ts/prod...,9.020.000đ,7.900.000đ ...,YES,Trả góp 0%,4,19
164,28255417,28255418,6576662490250,Card Màn Hình VGA ASUS DUAL-RTX2060S-A8G-EVO G...,Asus,https://tiki.vn//card-man-hinh-vga-asus-dual-r...,https://salt.tikicdn.com/cache/280x280/ts/prod...,13.453.000đ,11.790.000đ ...,YES,Trả góp 0%,4,20
